In [1]:
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import os
from keras.datasets.cifar100 import load_data
import tensorflow.contrib as tf_contrib


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar100.load_data(label_mode='fine')

In [3]:
# Data Normalization
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std
    return X_train, X_test
x_train, x_test = normalize(x_train, x_test)

# Data를 Shuffle해줌
seed = 777
np.random.seed(seed)
np.random.shuffle(x_train)
np.random.seed(seed)
np.random.shuffle(y_train)

In [4]:
# Convolutional Layer 함수
def conv(x, channels, kernel=4, stride=2, padding='SAME', use_bias=True, scope='conv_0'):
    with tf.variable_scope(scope):
        x = tf.layers.conv2d(inputs=x, filters=channels,
                             kernel_size=kernel, kernel_initializer=weight_init,
                             strides=stride, use_bias=use_bias, padding=padding)
    return x
# ResNet block Layers 함수 구현
def resblock(x_init, channels, is_training=True, use_bias=True, downsample=False, scope='resblock') :
    with tf.variable_scope(scope) :
        x = batch_norm(x_init, is_training, scope='batch_norm_0')
        x = tf.nn.relu(x)
        if downsample :
            x = conv(x, channels, kernel=3, stride=2, use_bias=use_bias, scope='conv_0')
            x_init = conv(x_init, channels, kernel=1, stride=2,
                          use_bias=use_bias, scope='conv_init')
        else :
            x = conv(x, channels, kernel=3, stride=1,
                     use_bias=use_bias, scope='conv_0')
        x = batch_norm(x, is_training, scope='batch_norm_1')
        x = tf.nn.leaky_relu(x)
        x = conv(x, channels, kernel=3, stride=1, use_bias=use_bias, scope='conv_1')

        return x + x_init

    return x
# Batch Normalization 함수 구현
def batch_norm(x, is_training=True, scope='batch_norm'):
    return tf_contrib.layers.batch_norm(x,
                                        decay=0.9, epsilon=1e-05,
                                        center=True, scale=True, updates_collections=None,
                                        is_training=is_training, scope=scope)


In [5]:
# Hyper Parameters Settings
learning_rate = 0.0015
num_epochs = 2
batch_size = 200
num_display = 200
weight_init = tf_contrib.layers.xavier_initializer_conv2d()

In [6]:
# Model Networks 구현

def get_model(X, by, is_training=True, reuse=False):
    with tf.variable_scope("network", reuse=reuse):
        residual_list = [2, 2, 2, 3]
#         residual_list = [3,4,6,3]
        ch = 32 # paper is 64
        x = conv(X, channels=ch, kernel=3, stride=1, scope='conv')

        for i in range(residual_list[0]) :
            x = resblock(x, channels=ch, is_training=is_training,
                         downsample=False, scope='resblock0_' + str(i))

        x = resblock(x, channels=ch*2, is_training=is_training,
                     downsample=True, scope='resblock1_0')

        for i in range(1, residual_list[1]) :
            x = resblock(x, channels=ch*2, is_training=is_training,
                         downsample=False, scope='resblock1_' + str(i))

        x = resblock(x, channels=ch*4, is_training=is_training,
                     downsample=True, scope='resblock2_0')

        for i in range(1, residual_list[2]) :
            x = resblock(x, channels=ch*4, is_training=is_training, downsample=False, scope='resblock2_' + str(i))

        x = tf.layers.dropout(x, 0.7 ,training=training)
        x = resblock(x, channels=ch*8, is_training=is_training, downsample=True, scope='resblock_3_0')

        for i in range(1, residual_list[3]) :
            x = resblock(x, channels=ch*8, is_training=is_training, downsample=False, scope='resblock_3_' + str(i))
        x = batch_norm(x, is_training, scope='batch_norm')
        x = tf.nn.relu(x)
        x=tf.layers.average_pooling2d(x, pool_size=2, strides=2, padding='SAME')
        flat = tf.reshape(x,
                          (-1, x.shape[1]*x.shape[2]*x.shape[3]))
        with tf.variable_scope('dense'):
            dense1 = tf.layers.dense(flat, 386,
                                  kernel_initializer=weight_init)
            dense_h = tf.nn.leaky_relu(dense1)
            dense_dropout = tf.layers.dropout(dense_h,0.5)
#             dense2 = tf.layers.dense(dense_dropout, 128,
#                                   kernel_initializer=weight_init)
#             dense_h = tf.nn.relu(dense2)
#             dense3 = tf.layers.dropout(dense_h,0.3)
            outputs = tf.layers.dense(dense_dropout, 100)
        one_hot = tf.squeeze(tf.one_hot(by, 100),axis=1) #이 부분을 tf.squeeze로 바꾸고 난뒤 학습이 되기 시작함.
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, 
                                                          labels=one_hot))
        opt = tf.train.AdamOptimizer(learning_rate=learning_rate,).minimize(loss)
        softmax = tf.nn.softmax(outputs)
        preds = tf.argmax(softmax, axis=1)
        acc = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(by, axis=1), preds), tf.float32))
        init = tf.global_variables_initializer()
    return {
        'loss': loss,
        'opt': opt,
        'preds': preds,
        'acc': acc,
        'init': init,
    }


In [7]:
X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3],name = 'input_data')
by = tf.placeholder(tf.int64, name ='y_input_data')
training = tf.placeholder(tf.bool, name = 'training_bool')
model = get_model(X, by, True)

In [8]:
with tf.Session() as sess:
    sess.run(model['init'])
    for ind_epoch in range(0, num_epochs):
        print('Current iteration {}'.format(ind_epoch + 1))
        
        for ind_ in range(0, int(50000 / batch_size)):
            batch_X = x_train[ind_*batch_size:(ind_+1)*batch_size]
            batch_by = y_train[ind_*batch_size:(ind_+1)*batch_size]
            _, cur_loss, cur_acc = sess.run(
                [model['opt'], model['loss'], model['acc']],
                feed_dict={X: batch_X, by: batch_by, training :True})
#             print(sess.run(model['preds'], feed_dict={X:x_train[:10], training:False}))
            if ind_ % num_display == 0:
                print('loss {0:.4f} acc {1:.4f}'.format(cur_loss, cur_acc))
    cur_acc_all = 0.0
    cur_loss_all = 0.0
    for ind_ in range(0, 10):
        cur_loss, cur_acc = sess.run(
                    [model['loss'], model['acc']],
                    feed_dict={X: x_test[ind_*1000:(ind_+1)*1000], 
                               by: y_test[ind_*1000:(ind_+1)*1000],
                              training : True})
        cur_loss_all += cur_loss
        cur_acc_all += cur_acc
    print('TEST: loss {0:.4f} acc {1:.4f}'.format(cur_loss_all / 10.0, 
                                              cur_acc_all / 10.0))
    print('LR',learning_rate)

Current iteration 1
loss 4.6746 acc 0.0050
loss 3.6870 acc 0.1500
Current iteration 2
loss 3.5895 acc 0.1450
loss 3.1994 acc 0.2400
TEST: loss 2.8887 acc 0.2754
LR 0.0015


Current iteration 1
loss 4.6968 acc 0.0067
loss 4.3137 acc 0.0400
Current iteration 2
loss 4.2011 acc 0.0400
loss 3.7758 acc 0.1067
TEST: loss 3.6424 acc 0.1341
LR 0.0001



Current iteration 1   

loss 4.6909 acc 0.0133
loss 3.8374 acc 0.1067
Current iteration 2
loss 3.8082 acc 0.1067
loss 3.4199 acc 0.1533
TEST: loss 3.1722 acc 0.2228